In [1]:
import numpy as np
import rasterio
from czml3 import CZML_VERSION, Document, Packet
from czml3.properties import (
    Color,
    Material,
    Polygon,
    PositionList,
    SolidColorMaterial,
)

import czml3_ext
from czml3_ext import packets, rasters
from czml3_ext.colours import (
    RGBA_black,
    RGBA_white,
    create_palette,
)

In [2]:
print(f"Using czml3-ext version: {czml3_ext.__version__}")

Using czml3-ext version: 0.11.0


## Coverage

### Coverage of a single sensor

In [3]:
vs = packets.coverage(rasters.ops("data.tif", 5), delete_rasters=True)

In [4]:
with rasterio.open("data.tif") as src:
    arr = src.read(1)
    origin_x, origin_y = src.transform * (0, 0)
    delta_x = src.transform[0]
    delta_y = src.transform[4]

In [5]:
sensor_coverage = packets.sensor(
    np.array(
        [
            [origin_y + arr.shape[0] / 2 * delta_y],
            [origin_x + arr.shape[1] / 2 * delta_x],
            [0],
        ]
    ),
    320,
    20,
    180,
    20,
    10_000,
)

In [6]:
doc = Document(
    packets=(
        [Packet(name="example", id="document", version=CZML_VERSION)]
        + vs
        + sensor_coverage
    )
)
with open("example.czml", "w") as f:
    f.write(doc.dumps())

### Coverage with a hole

In [7]:
vs_hole = packets.coverage(
    rasters.ops("data.tif", 5), rasters.ops("data_cut.tif", 5), delete_rasters=True
)

In [8]:
doc = Document(
    packets=(
        [Packet(name="example", id="document", version=CZML_VERSION)]
        + vs_hole
        + sensor_coverage
    )
)
with open("example.czml", "w") as f:
    f.write(doc.dumps())

### Coverage above a height
Show areas that are above 100 metres.

In [9]:
vs_height = packets.coverage(
    rasters.ops("data.tif", 100, "ge", band=2), delete_rasters=True
)

In [10]:
doc = Document(
    packets=(
        [Packet(name="example", id="document", version=CZML_VERSION)]
        + vs_height
        + sensor_coverage
    )
)
with open("example.czml", "w") as f:
    f.write(doc.dumps())

### Coverage between heights
Show areas that are between 100 metres and 200 metres. The method below uses a hole to remove areas below 100 metres.

In [11]:
vs_heights = packets.coverage(
    rasters.ops("data.tif", 200, "le", band=2),
    rasters.ops("data.tif", 100, "le", band=2),
    delete_rasters=True,
)

In [12]:
doc = Document(
    packets=(
        [Packet(name="example", id="document", version=CZML_VERSION)]
        + vs_heights
        + sensor_coverage
    )
)
with open("example.czml", "w") as f:
    f.write(doc.dumps())

The next method chains operations on the same raster and therefore doesn't require inputting holes to `packets.coverage()`. This is the recommended approach as it is usually 

In [13]:
vs_heights = packets.coverage(
    rasters.ops(
        "data.tif", [200, 100], ["le", "ge"], band=2
    ),  # same as rasters.ops("data.tif", [100, 200], ["ge", "le"], band=2)
    delete_rasters=True,
)

In [14]:
doc = Document(
    packets=(
        [Packet(name="example", id="document", version=CZML_VERSION)]
        + vs_heights
        + sensor_coverage
    )
)
with open("example.czml", "w") as f:
    f.write(doc.dumps())

`rasters.ops()` will raise an error if the output raster is empty. This can be turned off using the `error_if_no_data` input.

In [15]:
try:
    rasters.ops("data.tif", -999)  # error_if_no_data=True by default
except ValueError as e:
    print(e)

Created raster is empty.


### Amount of coverage overlap
Show polygons of the amount of overlap. The amount of overlap corresponds to a colour on a predefined scale.

In [16]:
fpaths = ["data_cut.tif", "data.tif"]
fpath_coverage, num_coverages = rasters.coverage_amount(fpaths, 5)

In [17]:
rgba = create_palette([RGBA_black, RGBA_white], len(fpaths) + 1)

In [18]:
overlaps = []
for num_covers in num_coverages:
    overlaps.extend(
        packets.coverage(
            rasters.ops(fpath_coverage, num_covers),
            name=f"Overlap of {num_covers}",
            polygon=Polygon(
                positions=PositionList(cartographicDegrees=[0, 0, 0]),
                material=Material(
                    solidColor=SolidColorMaterial(color=Color(rgba=rgba[num_covers]))
                ),
            ),
            delete_rasters=True,
        )
    )

In [19]:
doc = Document(
    packets=([Packet(name="example", id="document", version=CZML_VERSION)] + overlaps)
)
with open("example.czml", "w") as f:
    f.write(doc.dumps())